#### Problem Description

You have been asked to help a candidate to become US president. The winner of the election will be the candidate winning the most grand electors.
Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win (roughly, but not exactly, proportional to the size of the state) and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.
Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. We assume here there are only 2 candidates, and no history (no trend of certain states to vote for a particular candidate or party). Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.
You are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state.

#### Objective

You are asked to identify the states that should be prioritized to win the election, with a smart but simple algorithm (brute force computation of all possibilities to find the absolute optimum is not accepted, as it would be to computationally expensive). It is ok not to find the overall optimum, we just want a strategy that makes sense.
(This is of course a very simplistic view of reality, but it is a nice starting point to play with data and analyze possibilities).
First take a few minutes to think about what you need to do. Steps will be suggested hereafter, but a big part of the data scientist's job is to identify the flow of operations when being asked a practical question, so it is important you start exercising on that in addition to programming.

#### Data understanding

Here is what we are suggesting to do: we will rank states by decreasing number of grand electors per capita. The first states in the list will be the most valuable (you get a large number of grand electors by convincing a small number of people to vote for you). We will target all the states at the top of the list until the cumulative sum (also called running total) of grand electors won is larger than half the total number of Grand Electors in the country.

To do that, we need (you are allowed to create as many intermediary tables as you want, to keep queries short):
To join the 2 tables:
You notice States are not capitalized the same way in both tables (one is in uppercase letters, the other not), so you will first need to convert all to uppercase, for instance.
Now you can join the tables on the state key.
Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.
To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.
To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
To compute the running total of Grand Electors in that sorted list.

#### Hints

You can get inspiration from here to compute a running total from here:  https://stackoverflow.com/questions/21382766/cumulative-summing-values-in-sqliteLinks to an external site.
Independently, to compute the half of the total of Grand Electors overall (in the whole country):
This is the threshold we need to reach for winning the presidential election.
To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.

Hint: You can do that in 2 steps:
Select all the states for which the running total is below or equal to the threshold.
Add the first state for which the running total is larger than the threshold.
Can you draw some conclusions from the result? Is it in line with your expectations? How many states do you end up with in the target list? Is it a small or a large number? Do you think it would be a good recommendation to target those states?


#### Importing the data

In [1]:
import pandas as pd

In [2]:
grandElectors = pd.read_csv('GrandElectors_by_state.csv')
grandElectors['State'] = grandElectors['State'].str.upper()
grandElectors.head(5)

,State,GrandElectors
0,ALABAMA,9
1,ALASKA,3
2,ARIZONA,11
3,ARKANSAS,6
4,CALIFORNIA,55


In [3]:
pop = pd.read_csv('Population_by_state.csv')
pop.head(5)

,State,Population
0,CALIFORNIA,39144818
1,TEXAS,27469114
2,FLORIDA,20271272
3,NEW YORK,19795791
4,ILLINOIS,12859995


#### Merging two tables

In [4]:
mainDF = pop.merge(grandElectors, on=['State'])
mainDF = mainDF.replace('DISTRICT OF COLUMBIA', 'DC')
mainDF

,State,Population,GrandElectors
0,CALIFORNIA,39144818,55
1,TEXAS,27469114,38
2,FLORIDA,20271272,29
3,NEW YORK,19795791,29
4,ILLINOIS,12859995,20
5,PENNSYLVANIA,12802503,20
6,OHIO,11613423,18
7,GEORGIA,10214860,16
8,NORTH CAROLINA,10042802,15
9,MICHIGAN,9922576,16


In [5]:
# adding Ration column

mainDF['Ratio'] = mainDF['GrandElectors'] / mainDF['Population']
mainDF.sort_values(by='Ratio', ascending=False,inplace=True)
mainDF

,State,Population,GrandElectors,Ratio
50,WYOMING,586107,3,0.000005
49,VERMONT,626042,3,0.000005
48,DC,672228,3,0.000004
47,ALASKA,738432,3,0.000004
46,NORTH DAKOTA,756927,3,0.000004
42,RHODE ISLAND,1056298,4,0.000004
45,SOUTH DAKOTA,858469,3,0.000003
44,DELAWARE,945934,3,0.000003
41,MAINE,1329328,4,0.000003
40,NEW HAMPSHIRE,1330608,4,0.000003


In [6]:
# adding Ru ning total for each GrandElectors for State

mainDF['GElectorsRunningTotal'] = mainDF['GrandElectors'].cumsum()
mainDF

,State,Population,GrandElectors,Ratio,GElectorsRunningTotal
50,WYOMING,586107,3,0.000005,3
49,VERMONT,626042,3,0.000005,6
48,DC,672228,3,0.000004,9
47,ALASKA,738432,3,0.000004,12
46,NORTH DAKOTA,756927,3,0.000004,15
42,RHODE ISLAND,1056298,4,0.000004,19
45,SOUTH DAKOTA,858469,3,0.000003,22
44,DELAWARE,945934,3,0.000003,25
41,MAINE,1329328,4,0.000003,29
40,NEW HAMPSHIRE,1330608,4,0.000003,33


#### Identifying the threshold 



In [16]:
value = int(mainDF.GrandElectors.sum()) / 100 * 51
value

274.38

In [17]:
df = mainDF.query('GElectorsRunningTotal <= @value')
df

,State,Population,GrandElectors,Ratio,GElectorsRunningTotal
50,WYOMING,586107,3,0.000005,3
49,VERMONT,626042,3,0.000005,6
48,DC,672228,3,0.000004,9
47,ALASKA,738432,3,0.000004,12
46,NORTH DAKOTA,756927,3,0.000004,15
42,RHODE ISLAND,1056298,4,0.000004,19
45,SOUTH DAKOTA,858469,3,0.000003,22
44,DELAWARE,945934,3,0.000003,25
41,MAINE,1329328,4,0.000003,29
40,NEW HAMPSHIRE,1330608,4,0.000003,33


#### Adding the first state for which the running total is larger than the threshold

In [20]:
import warnings
warnings.filterwarnings('ignore') 

df.loc[len(mainDF.index)] = ['GEORGIA', 10214860, 16, 0.000002, 287]
df['GElectorsRunningTotal'] = df['GrandElectors'].cumsum()
df

,State,Population,GrandElectors,Ratio,GElectorsRunningTotal
50,WYOMING,586107,3,0.000005,3
49,VERMONT,626042,3,0.000005,6
48,DC,672228,3,0.000004,9
47,ALASKA,738432,3,0.000004,12
46,NORTH DAKOTA,756927,3,0.000004,15
42,RHODE ISLAND,1056298,4,0.000004,19
45,SOUTH DAKOTA,858469,3,0.000003,22
44,DELAWARE,945934,3,0.000003,25
41,MAINE,1329328,4,0.000003,29
40,NEW HAMPSHIRE,1330608,4,0.000003,33
